In [68]:
from torch.utils.data import Dataset
import numpy as np
from torch.nn.functional import one_hot
import torch

class M100ImageDataset(Dataset):
    def __init__(self, *, data_path, train=True):
        # Loading data from the 'mnist_compressed.npz' file
        data = np.load(data_path)
        # Reading variables containing the data
        X_test, y_test, X_train, y_train =  data['test_images'], data['test_labels'], data['train_images'], data['train_labels']

        if train:
            self.data = torch.tensor(X_train.astype('float32') / 255., dtype=torch.float32).unsqueeze(1)
            self.labels = one_hot(torch.tensor(y_train), num_classes=100).to(torch.float32)
        else:
            self.data = torch.tensor(X_test.astype('float32') / 255., dtype=torch.float32).unsqueeze(1)
            self.labels = one_hot(torch.tensor(y_test), num_classes=100).to(torch.float32)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]

In [69]:
from torch.utils.data import DataLoader

m100_path = '../data/mnist100/mnist_compressed.npz'

training_data = M100ImageDataset(data_path=m100_path, train=True)
test_data = M100ImageDataset(data_path=m100_path, train=False)

train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

In [70]:
import torch.nn as nn

class M100Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, stride=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=(5*12*16), out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=100)
        self.softmax = nn.LogSoftmax()

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.softmax(x)

In [71]:
allow_cuda = False


use_cuda = torch.cuda.is_available()
device = torch.device('cuda') if use_cuda and allow_cuda else torch.device('cpu')

loss_fn = torch.nn.SmoothL1Loss()

model = M100Net()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [78]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        #print(inputs.shape)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
print("Finished Training")

/tmp/ipykernel_255393/3076540559.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(x)


[1,   100] loss: 0.206
[1,   200] loss: 0.206
[1,   300] loss: 0.206
[1,   400] loss: 0.206
[1,   500] loss: 0.206
[1,   600] loss: 0.206
[1,   700] loss: 0.206
[1,   800] loss: 0.206
[1,   900] loss: 0.206
[1,  1000] loss: 0.206
[1,  1100] loss: 0.206
[1,  1200] loss: 0.206
[1,  1300] loss: 0.206
[1,  1400] loss: 0.206
[1,  1500] loss: 0.206
[1,  1600] loss: 0.206
[1,  1700] loss: 0.206
[1,  1800] loss: 0.206
[2,   100] loss: 0.206
[2,   200] loss: 0.206
[2,   300] loss: 0.206
[2,   400] loss: 0.206
[2,   500] loss: 0.206
[2,   600] loss: 0.206
[2,   700] loss: 0.206
[2,   800] loss: 0.206
[2,   900] loss: 0.206
[2,  1000] loss: 0.206
[2,  1100] loss: 0.206
[2,  1200] loss: 0.206
[2,  1300] loss: 0.206
[2,  1400] loss: 0.206
[2,  1500] loss: 0.206
[2,  1600] loss: 0.206
[2,  1700] loss: 0.206
[2,  1800] loss: 0.206
Finished Training


In [74]:
save_path = "../saved_models/test.pth"
torch.save(model.state_dict(), save_path)

In [95]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        labels = torch.argmax(labels, dim=1)
        total += labels.size(0)
        for j in range(labels.shape[0]):
            if predicted[j] == labels[j]:
                correct += 1
        
        #correct += (predicted == labels).sum().item()

print(correct)
print(total)
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

/tmp/ipykernel_255393/3076540559.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(x)


85
10000
Accuracy of the network on the 10000 test images: 0 %
